# Theory

## Forward-Propagation

$$
X \rightarrow Z=WX+b_1 \rightarrow H=\sigma(Z) \rightarrow U = CH + b_2\rightarrow S=F_{softmax}(U) \rightarrow \rho(S, y) = \log S_y,
$$
where $S_y=\frac{\exp(U_y)}{\sum_{j=0}^{K-1}\exp(U_j)}$ is the y-th element of the $S$ and $U_y$ is the y-th element of the $U$.

## Backward-Propagation

$$
\frac{\partial \rho}{\partial U_t} =  
\begin{cases}
S_t(U), & t\neq y \\
1- S_t(U). & t = y
\end{cases}
\Longrightarrow
\frac{\partial \rho}{\partial U} = e_y - S(U), 
$$

where $e_y$ is the unit vector, which y-th coordinate equals to 1 and 0 elsewhere. 

\begin{align*}
& \frac{\partial \rho}{\partial b_2} = \frac{\partial \rho}{\partial U}\frac{\partial U}{\partial b_2} = e_y - S(U) \\
& \frac{\partial \rho}{\partial C} = \frac{\partial \rho}{\partial U}\frac{\partial U}{\partial C} = (e_y - S(U))H^T \\
& \frac{\partial \rho}{\partial H} = \frac{\partial \rho}{\partial U}\frac{\partial U}{\partial H} = C^T\frac{\partial \rho}{\partial U} =C^T(e_y - S(U)) \\
& \frac{\partial \rho}{\partial b_1} = \frac{\partial \rho}{\partial H}\frac{\partial H}{\partial Z}\frac{\partial Z}{\partial b_1} = \frac{\partial \rho}{\partial H} \odot \sigma'(Z)\\
& \frac{\partial \rho}{\partial W} =  \big(\frac{\partial \rho}{\partial H} \odot \sigma'(Z)\big)X^T
\end{align*}

## Algorithm
Mini-Batch Stochastic gradient algorithm for updating $\theta = \{W, b_1, C, b_2\}$:
* Step1: Specify batch_size $M$, activation function $\sigma(z)$, and initialize $W^{(0)}, b_1^{(0)}, C^{(0)}, b_2^{(0)}$;
* Step2: At iteration $t$:
    * a. Select $M$ data samples $\{X^{(t,m)},y^{(t,m)}\}_{m=1}^M$ uniform at random from the full dataset $\{X^{(n)},y^{(n)}\}_{n=1}^N$ 
    * b. Compute forward-propagation:
        * $Z^{(t,m)}=W^{(t)}X^{(t,m)}+b_1^{(t)}$
        * $H^{(t,m)}=\sigma(Z^{(t,m)})$
        * $U^{(t,m)} = C^{(t)}H^{(t,m)} + b_2^{(t)}$
        * $S^{(t,m)}=F_{softmax}(U^{(t,m)})$
    * c. Compute backward-propagation:
        * $\frac{\partial \rho}{\partial b_2} = \frac{1}{M}\sum_{m=1}^M e_{y^{(t,m)}} - S^{(t,m)}$ 
        * $\frac{\partial \rho}{\partial C} =   \frac{1}{M}\sum_{m=1}^M  (e_{y^{(t,m)}}  - S^{(t,m)}){H^{(t,m)}}^T$
        * $\frac{\partial \rho}{\partial H} = \frac{1}{M}\sum_{m=1}^M C^T(e_{y^{(t,m)}} - S^{(t,m)})$
        * $\frac{\partial \rho}{\partial b_1} = \frac{1}{M}\sum_{m=1}^M \frac{\partial \rho}{\partial H} \odot \sigma'(Z^{(t,m)})$
        * $\frac{\partial \rho}{\partial W} =  \frac{1}{M}\sum_{m=1}^M \big(\frac{\partial \rho}{\partial H} \odot \sigma'(Z^{(t,m)})\big){X^{(t,m)}}^T$
    * Given learning rate $\eta_t$, update parameters as follows:
        * $b_2^{(t+1)}) \leftarrow b_2^{(t)}) + \eta_t \frac{\partial \rho}{\partial b_2}$
        * $C^{(t+1)}) \leftarrow C^{(t)}) + \eta_t \frac{\partial \rho}{\partial C}$
        * $b_1^{(t+1)}) \leftarrow b_1^{(t)}) + \eta_t \frac{\partial \rho}{\partial b_1}$
        * $W^{(t+1)}) \leftarrow W^{(t)}) + \eta_t \frac{\partial \rho}{\partial W}$
* Step3: Repeat Step2 until some convergence criteria is met.

To avoid unnecessary `for-loop` we can vectoruize the above algorithm. 

* Step1: Specify batch_size $M$, activation function $\sigma(z)$, and initialize $W^{(0)}, b_1^{(0)}, C^{(0)}, b_2^{(0)}$;
* Step2: At iteration $t$:
    * a. Select $M$ data samples $\{X^{(t,m)},y^{(t,m)}\}_{m=1}^M$ uniform at random from the full dataset $\{X^{(n)},y^{(n)}\}_{n=1}^N$ 
    * b. Compute forward-propagation:
        * $Z^{(t)}=W^{(t)}X^{(t)}+b_1^{(t)}$, where $X^{(t)} = (X^{(t,1)},...,X^{(t,M)})$ and the summation on $b_1$ will be column-wise.
        * $H^{(t)}=\sigma(Z^{(t)})$, where $H^{(t)} = (H^{(t,1)},...,H^{(t,M)})$ and $\sigma(.)$ is element wise operation.
        * $U^{(t)} = C^{(t)}H^{(t)} + b_2^{(t)}$
        * $S^{(t)}=F_{softmax}(U^{(t)})$, where the $F_{softmax}$ is column-wise operation.
    * c. Compute backward-propagation:
        * $\frac{\partial \rho}{\partial b_2} = \text{np.mean}(e_{y^{(t)}} - S^{(t)}, \text{axis=1})$
        * $\frac{\partial \rho}{\partial C} =  \frac{1}{M} (e_{y^{(t)}}  - S^{(t)}){H^{(t)}}^T$
        * $\frac{\partial \rho}{\partial H} = \text{np.mean}(C^T(e_{y^{(t)}} - S^{(t)}), \text{axis=1})$
        * $\frac{\partial \rho}{\partial b_1} = \text{np.mean}(\frac{\partial \rho}{\partial H} \odot \sigma'(Z^{(t)}), \text{axis=1})$
        * $\frac{\partial \rho}{\partial W} =  \frac{1}{M}(\big(\frac{\partial \rho}{\partial H} \odot \sigma'(Z^{(t)})\big){X^{(t)}}^T)$
    * Given learning rate $\eta_t$, update parameters as follows:
        * $b_2^{(t+1)}) \leftarrow b_2^{(t)}) + \eta_t \frac{\partial \rho}{\partial b_2}$
        * $C^{(t+1)}) \leftarrow C^{(t)}) + \eta_t \frac{\partial \rho}{\partial C}$
        * $b_1^{(t+1)}) \leftarrow b_1^{(t)}) + \eta_t \frac{\partial \rho}{\partial b_1}$
        * $W^{(t+1)}) \leftarrow W^{(t)} + \eta_t \frac{\partial \rho}{\partial W}$
* Step3: Repeat Step2 until some convergence criteria is met.

# Numerical Experiment

In [5]:
import numpy as np
import h5py
import time 
import copy
#import logging
#from helperfunctions import create_log
#logger = create_log(file_name="task.log", log_level=logging.DEBUG)
file_name = "../data/MNISTdata.hdf5"

In [6]:
#logger.info("Load the MNIST dataset...")
data = h5py.File(file_name, "r")
x_train = np.float32(data["x_train"][:])
y_train = np.int32(np.hstack(np.array(data["y_train"])))
x_test = np.float32(data["x_test"][:])
y_test = np.int32(np.hstack(np.array(data["y_test"])))
data.close()
#logger.info("Finished!")

In [3]:
class MnistModel():
    def __init__(self, x_train, y_train, x_test, y_test, hidden_units=100, learning_rate=0.01, batch_size=20, num_epochs=5, seed=None):
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
        self.num_inputs = self.x_train.shape[1]
        self.num_outputs = 10
        self.hidden_units = hidden_units
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.params = {}
        self.gradients = {}
        if seed is not None:
            r = np.random.RandomState(seed)
            self.params["W"] = r.randn(self.hidden_units, self.num_inputs) / np.sqrt(self.num_inputs)
            self.params["b1"] = np.zeros((self.hidden_units, 1))  
            self.params["C"] = r.randn(self.num_outputs, self.hidden_units) / np.sqrt(self.num_inputs)
            self.params["b2"] = np.zeros((self.num_outputs, 1)) 
        else:
            self.params["W"] = np.random.randn(self.hidden_units, self.num_inputs) / np.sqrt(self.num_inputs)
            self.params["b1"] = np.zeros((self.hidden_units, 1))
            self.params["C"] = np.random.randn(self.num_outputs, self.hidden_units) / np.sqrt(self.num_inputs)
            self.params["b2"] = np.zeros((self.num_outputs, 1))
        print("training sample size: [{}]\ntest sample size:[{}]\nhidden units number: [{}]\nbatch_size:[{}]".format(self.x_train.shape, self.x_test.shape, self.hidden_units, self.batch_size))

    def activation(self, z):
        """
        z: must be of size (hidden_units * 1)
        """
        return [*map(lambda x: x if x > 0 else 0, z)]

    def activation_gradient(self, z):
        """
        z: must be of size (hidden_units * 1)
        """
        return [*map(lambda x: 1 if x > 0 else 0, z)]

    def softmax(self, U):
        temp = np.exp(U)
        return temp / np.sum(temp)


    def forward_propagation(self):
        random_index = np.random.choice(self.x_train.shape[0], replace=False, size=self.batch_size)
        self.x_train_sub_samples = self.x_train[random_index].reshape((-1, self.batch_size))
        self.y_train_sub_samples = self.y_train[random_index]
        self.forward_results = {}
        self.forward_results["Z"] = np.dot(self.params["W"], self.x_train_sub_samples) + self.params["b1"]
        self.forward_results["H"] = np.apply_along_axis(self.activation, 0, self.forward_results["Z"])
        self.forward_results["U"] = np.dot(self.params["C"], self.forward_results["H"]) + self.params["b2"]
        self.forward_results["S"] = np.apply_along_axis(self.softmax, 0, self.forward_results["U"])

    def create_unit_matrix(self):
        ey = np.zeros((self.num_outputs, self.batch_size))
        for col_index, row_index in enumerate(self.y_train_sub_samples):
            ey[row_index, col_index] = 1
        return(ey)

    def back_propagation(self):
        ey = self.create_unit_matrix()
        temp = - (ey - self.forward_results["S"])
        self.gradients["db2"] = np.mean(temp, axis=1, keepdims=True)
        self.gradients["dC"] = np.dot(temp, self.forward_results["H"].T) / self.batch_size
        self.gradients["dH"] = np.mean(np.dot(self.params["C"].T, temp), axis=1, keepdims=True)
        H_gradient = np.apply_along_axis(self.activation_gradient, 0, self.forward_results["Z"])
        temp2 = np.multiply(self.gradients["dH"], H_gradient)
        self.gradients["db1"] = np.mean(temp2, axis=1, keepdims=True)
        self.gradients["dW"] = np.dot(temp2, self.x_train_sub_samples.T) / self.batch_size

    def train(self):
        for epoch in range(self.num_epochs):
            if (epoch > 5):
                self.learning_rate = 0.001
            if (epoch > 10):
                self.learning_rate = 0.0001
            if (epoch > 15):
                self.learning_rate = 0.00001
            total_correct = 0
            for i in range(int(self.x_train.shape[0] / self.batch_size)):
                self.forward_propagation()
                prediction_train =  np.argmax(self.forward_results["S"], axis=0)
                total_correct += np.sum(prediction_train == self.y_train_sub_samples)
                self.back_propagation()
                self.params["W"] -= self.learning_rate * self.gradients["dW"]
                self.params["b1"] -= self.learning_rate * self.gradients["db1"]
                self.params["C"] -= self.learning_rate * self.gradients["dC"]
                self.params["b2"] -= self.learning_rate * self.gradients["db2"]
            print("epoch:{} | Training Accuracy:[{}]".format(epoch+1, total_correct/len(self.x_train)))
    def test(self):
        self.Z = np.dot(self.params["W"], self.x_test.T) + self.params["b1"]
        self.H = np.apply_along_axis(self.activation, 0, self.Z)
        self.U = np.dot(self.params["C"], self.H) + self.params["b2"]
        self.S = np.apply_along_axis(self.softmax, 0, self.U)
        self.prediction = np.apply_along_axis(np.argmax, 0, self.S)
        correct_ratio = np.mean(self.prediction == self.y_test)
        return correct_ratio

## Batch_Size = 1

In [4]:
nn = MnistModel(x_train, y_train, x_test, y_test, hidden_units=100, batch_size=1, learning_rate=0.01, num_epochs=5, seed=1234)

training sample size: [(60000, 784)]
test sample size:[(10000, 784)]
hidden units number: [100]
batch_size:[1]


In [5]:
start = time.time()
nn.train()
end = time.time()

epoch:1 | Training Accuracy:[0.9296]
epoch:2 | Training Accuracy:[0.96985]
epoch:3 | Training Accuracy:[0.9783333333333334]
epoch:4 | Training Accuracy:[0.9819166666666667]
epoch:5 | Training Accuracy:[0.98605]


In [6]:
print("Running Time: [{}] second".format(end - start))

Running Time: [672.2338092327118] second


In [8]:
print("Test Accuracy: [{}]".format(nn.test()))

Test Accuracy: [0.975]


## Bug happens when `batch_size>1`

In [47]:
class MnistModel():
    def __init__(self, x_train, y_train, x_test, y_test, hidden_units=100, learning_rate=0.01, batch_size=20, num_epochs=5, seed=None):
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
        self.num_inputs = self.x_train.shape[1]
        self.num_outputs = 10
        self.hidden_units = hidden_units
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.params = {}
        self.gradients = {}
        if seed is not None:
            r = np.random.RandomState(seed)
            self.params["W"] = r.randn(self.hidden_units, self.num_inputs) / np.sqrt(self.num_inputs)
            self.params["b1"] = np.zeros((self.hidden_units, 1))  
            self.params["C"] = r.randn(self.num_outputs, self.hidden_units) / np.sqrt(self.num_inputs)
            self.params["b2"] = np.zeros((self.num_outputs, 1)) 
        else:
            self.params["W"] = np.random.randn(self.hidden_units, self.num_inputs) / np.sqrt(self.num_inputs)
            self.params["b1"] = np.zeros((self.hidden_units, 1))
            self.params["C"] = np.random.randn(self.num_outputs, self.hidden_units) / np.sqrt(self.num_inputs)
            self.params["b2"] = np.zeros((self.num_outputs, 1))
        print("training sample size: [{}]\ntest sample size:[{}]\nhidden units number: [{}]\nbatch_size:[{}]".format(self.x_train.shape, self.x_test.shape, self.hidden_units, self.batch_size))

    def activation(self, z):
        """
        z: must be of size (hidden_units * 1)
        """
        return [*map(lambda x: x if x > 0 else 0, z)]

    def activation_gradient(self, z):
        """
        z: must be of size (hidden_units * 1)
        """
        return [*map(lambda x: 1 if x > 0 else 0, z)]

    def softmax(self, U):
        temp = np.exp(U)
        return temp / np.sum(temp)


    def forward_propagation(self):
        random_index = np.random.choice(self.x_train.shape[0], replace=False, size=self.batch_size)
        self.x_train_sub_samples = self.x_train[random_index].reshape((-1, self.batch_size))
        self.y_train_sub_samples = self.y_train[random_index]
        self.forward_results = {}
        self.forward_results["Z"] = np.dot(self.params["W"], self.x_train_sub_samples) + self.params["b1"]
        self.forward_results["H"] = np.apply_along_axis(self.activation, 0, self.forward_results["Z"])
        self.forward_results["U"] = np.dot(self.params["C"], self.forward_results["H"]) + self.params["b2"]
        self.forward_results["S"] = np.apply_along_axis(self.softmax, 0, self.forward_results["U"])

    def create_unit_matrix(self):
        ey = np.zeros((self.num_outputs, self.batch_size))
        for col_index, row_index in enumerate(self.y_train_sub_samples):
            ey[row_index, col_index] = 1
        return(ey)

    def back_propagation(self):
        ey = self.create_unit_matrix()
        temp = - (ey - self.forward_results["S"])
        self.gradients["db2"] = np.mean(temp, axis=1, keepdims=True)
        self.gradients["dC"] = np.dot(temp, self.forward_results["H"].T) / self.batch_size
        temp2 = np.dot(self.params["C"].T, temp)
        self.gradients["dH"] = np.mean(temp2, axis=1, keepdims=True)
        H_gradient = np.apply_along_axis(self.activation_gradient, 0, self.forward_results["Z"])
        temp3 = np.multiply(temp2, H_gradient)
        self.gradients["db1"] = np.mean(temp3, axis=1, keepdims=True)
        #self.gradients["dW"] = np.dot(temp3, self.x_train_sub_samples.T) / self.batch_size
        self.gradients["dW"] = 0
        for i in range(int(temp3.shape[1])):
            #print(temp3[i].reshape(-1,1).shape, self.x_train_sub_samples.shape)
            self.gradients["dW"] += np.dot(temp3[i].reshape(-1,1), self.x_train_sub_samples[:,i].reshape(1, self.num_inputs)) 
        self.gradients["dW"] /= self.batch_size

    def train(self):
        for epoch in range(self.num_epochs):
            if (epoch > 5):
                self.learning_rate = 0.001
            if (epoch > 10):
                self.learning_rate = 0.0001
            if (epoch > 15):
                self.learning_rate = 0.00001
            total_correct = 0
            for i in range(int(self.x_train.shape[0] / self.batch_size)):
                self.forward_propagation()
                prediction_train =  np.argmax(self.forward_results["S"], axis=0)
                total_correct += np.sum(prediction_train == self.y_train_sub_samples)
                self.back_propagation()
                #print(self.params["W"].shape, self.gradients["dW"].shape)
                self.params["W"] -= self.learning_rate * self.gradients["dW"]
                self.params["b1"] -= self.learning_rate * self.gradients["db1"]
                self.params["C"] -= self.learning_rate * self.gradients["dC"]
                self.params["b2"] -= self.learning_rate * self.gradients["db2"]
            print("epoch:{} | Training Accuracy:[{}]".format(epoch+1, total_correct/len(self.x_train)))
    def test(self):
        self.Z = np.dot(self.params["W"], self.x_test.T) + self.params["b1"]
        self.H = np.apply_along_axis(self.activation, 0, self.Z)
        self.U = np.dot(self.params["C"], self.H) + self.params["b2"]
        self.S = np.apply_along_axis(self.softmax, 0, self.U)
        self.prediction = np.apply_along_axis(np.argmax, 0, self.S)
        correct_ratio = np.mean(self.prediction == self.y_test)
        return correct_ratio

In [48]:
nn = MnistModel(x_train, y_train, x_test, y_test, hidden_units=100, batch_size=100, learning_rate=0.01, num_epochs=20, seed=1234)

training sample size: [(60000, 784)]
test sample size:[(10000, 784)]
hidden units number: [100]
batch_size:[100]


In [49]:
nn.train()

epoch:1 | Training Accuracy:[0.10266666666666667]
epoch:2 | Training Accuracy:[0.10678333333333333]
epoch:3 | Training Accuracy:[0.10898333333333333]


KeyboardInterrupt: 

In [2]:
a = np.array([[1,2],[3,4]])
b = np.array([[1],[2]])
np.multiply(a,b)

array([[1, 2],
       [6, 8]])